<a href="https://colab.research.google.com/github/zypchn/zeytin/blob/main/MedicalRAG_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! pip install -qU langchain langchain-pinecone langchain-openai langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [33]:
import os
from getpass import getpass
from huggingface_hub import notebook_login
from pinecone import ServerlessSpec, Pinecone as PineconeClient
import pandas as pd
from datasets import load_dataset
from langchain_community.document_loaders import TextLoader, CSVLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_pinecone import PineconeVectorStore
from uuid import uuid4

In [5]:
os.environ["LANGCHAIN_API_KEY"] = getpass("LANGCHAIN_API_KEY : ")
os.environ["LANGSMITH_API_KEY"] = getpass("LANGSMITH_API_KEY : ")
os.environ["OPENAI_API_KEY"] = getpass("OPENAI_API_KEY : ")
os.environ["PINECONE_API_KEY"] = getpass("PINECONE_API_KEY : ")
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
#notebook_login()

os.environ["LANGSMITH_TRACING_V2"]="true"
os.environ["LANGSMITH_PROJECT"]="zeytin"

LANGCHAIN_API_KEY : ··········
LANGSMITH_API_KEY : ··········
OPENAI_API_KEY : ··········
PINECONE_API_KEY : ··········


In [11]:
pth = "/content/diagnosis.csv"
loader = CSVLoader(file_path=pth,
                   csv_args={
                       "delimiter": ",",
                       "quotechar": '"',
                       "fieldnames": ["category", "topic", "text", "num_tokens"]
                   })

docs = loader.load()

In [16]:
pc = PineconeClient(api_key=pinecone_api_key)

In [42]:
index_name = "medicalrag-demo"

if not pc.has_index(index_name):
  pc.create_index(
      name=index_name,
      dimension=1536,
      metric="cosine",
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

index = pc.Index(index_name)

In [18]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [43]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [44]:
half_len = len(all_splits) // 2
uuids = [str(uuid4()) for _ in range(len(all_splits))]

try:
  vector_store.add_documents(documents=all_splits[:half_len], ids=uuids[:half_len])
  print("Successfully loaded the first half of the documents.")
  vector_store.add_documents(documents=all_splits[half_len:], ids=uuids[half_len:])
  print("Successfully loaded the second half of the documents.")
except Exception as e:
  print(e)

Successfully loaded the first half of the documents.
Successfully loaded the second half of the documents.


In [45]:
query = "Kan örneği alınarak yapılan testler hangileridir?"
results = vector_store.similarity_search_with_score(
    query, k=7
)

for res, score in results:
  print(f"SCORE : {score}\n{res.page_content}\n\n")

SCORE : 0.560539305
gibi hücrelerin sayılarak, çeşitli hastalıkların tespitinde kullanılan kan testidir. Hemogram testi ile anlaşılabilen sorunların başında başında enfeksiyon, anemi, bağışıklık sistemi hastalıkları ve kan kanseri gibi hastalıklar gelmektedir. Tam kan Sayımı, CBC, hemogram, diferansiyel ile CBC, TKS isimleriyle de bilinir.Hemogram (Tam Kan Sayımı) Testi Hangi Hastalıkların Tanısı İçin Yapılır?Kişiden kan örneği alınmasıyla yapılan tam kan sayımı (hemogram) testi ile anemi, otoimmün bozukluklar, kemik iliği bozuklukları, dehidrasyon, enfeksiyonlar, iltihaplanma, lösemi, lenfoma, miyeloproliferatif neoplazmlar, miyelodisplastik sendrom, orak hücre hastalığı, talasemi, B12 vitamin, folat ve demir eksikliği gibi vitamin eksiklikleri tespit edilebilir.Tam kan sayımı testinde şu hastalıklar tespit edilir:Hemogram (Tam Kan Sayımı) Testinde Hangi Değerler İncelenir?Test yaptırdıktan sonra size sunulan sonuç sayfanızda yer alan bir dizi alt parametre yer alır. Bu alt parametrel